**Model**

In [ ]:
#Imports

from pathlib import Path
import torch, os
import matplotlib.pyplot as plt
import torchvision.transforms as t
from torch import nn, optim

In [ ]:
# Data preprocessing
train_path = Path.cwd() / "train"
test_path = Path.cwd() / "test"

from turtle import color
from torch.utils.data import Dataset
from itertools import chain
from PIL import Image
from kornia.color import rgb_to_lab, lab_to_rgb
from skimage.color import lab2rgb, rgb2lab, rgb2gray
import numpy as np
#from natsort import natsorted

class Data(Dataset):
    def __init__(self, main_dir, transform):
         
        # Set the loading directory
        self.main_dir = main_dir
        self.transform = transform
        # List all images in folder and count them
        self.all_imgs = os.listdir(main_dir)
    
    def __len__(self):
        # Return the previously computed number of images
        return len(self.all_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.all_imgs[idx])

        # Use PIL for image loading
        img = Image.open(img_loc).convert("RGB")
        
        # Apply transformations
        image = self.transform(img)

        # Turn RBG -> LAB
        img_lab = rgb_to_lab(image)
        # Extract l and ab channels into sepparate tensors
        img_ab = img_lab[1:3,:,:]
        img_l = img_lab[0:1,:,:]

        return img_l, img_ab

from torchvision.transforms import Compose
transform = Compose([t.ToTensor(), t.CenterCrop(256)])

example_dataset = Data(train_path, transform)
l, ab = example_dataset[300]
print(len(example_dataset))


In [ ]:
def plot_comb(l_input, ab_input, img_size = 256):
    '''Show rgb image from grayscale and ab channels'''
    # Genrate ab for grayscale plot
    bw_ab = torch.zeros((2,img_size,img_size))

    # Combine channels
    color_image = torch.cat((l_input, ab_input), 0)
    bw_image = torch.cat((l_input, bw_ab), 0)
    
    # Lab -> RBG for plotting
    color_img = lab_to_rgb(color_image)
    bw_img = lab_to_rgb(bw_image)
    
    fig, axes = plt.subplots(1, 2, figsize=(20,10))
    img_data = color_img.permute(1, 2, 0)
    axes[0].imshow(img_data, cmap='gray')
    img_data = bw_img.permute(1, 2, 0)
    axes[1].imshow(img_data, cmap='gray')
    
def display_image(axis, image_tensor):
    """Display a tensor as an image
    
    Args:
        axis (pyplot axis)
        image_tensor (torch.Tensor): tensor with shape (num_channels=3, width, heigth)
    """
    
    # See hint above
    if not isinstance(image_tensor, torch.Tensor):
        raise TypeError("The `display_image` function expects a `torch.Tensor` " +
                        "use the `ToTensor` transformation to convert the images to tensors.")
        
    # The imshow commands expects a `numpy array` with shape (3, width, height)
    # We rearrange the dimensions with `permute` and then convert it to `numpy`
    image_data = image_tensor.permute(1, 2, 0)
    axis.imshow(image_data, cmap='gray')
    # By convention when working with images, the origin is at the top left corner.
    # Therefore, we switch the order of the y limits.

def display_comp(g_img, img):
    fig, axes = plt.subplots(1, 2, figsize=(20,10))
    display_image(axes[0], g_img)
    display_image(axes[1], img)
    plt.show()

# Example usage
some_random_index = 300
l, ab = example_dataset[some_random_index]
plot_comb(l, ab)
# display_comp(lab_to_rgb(g_img), lab_to_rgb(c_img))

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32
train_dataset = Data(train_path, transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = Data(test_path, transform)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision.models as models

## CNN MODULE
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding = 1), 
            nn.ReLU(),
            nn.BatchNorm2d(64)
        )

        self.layer_2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding = 1), 
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )

        self.layer_3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding = 1), 
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )

        self.layer_4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=2, kernel_size=3, padding = 1)
        )
        
    def forward(self, input_batch):
        x = self.layer_1(input_batch)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        return x

model = CNN()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0)

In [ ]:
l, ab = example_dataset[some_random_index]
l_batch = torch.unsqueeze(l, 0)
ab_res = model.forward(l_batch).detach()
ab_res = torch.squeeze(ab_res, dim = 0)
plot_comb(l, ab_res)

In [ ]:
# TODO implement CUDA GPU training

for epoch in range(5):
    # Compute predictions and back-prop in the training set
    losses = []
    n_correct = 0
    batch = 0
    for b_x, b_y in train_dataloader:
        pred = model(b_x)
        loss = loss_fn(pred, b_y)
        losses.append(loss.item())
        batch += 1
        if batch%50 == 0: 
            print(f'{batch} batches trained')
            
        # if iter == 60: 
        #     break
        
        
        #hard_preds = pred.argmax(dim=1)
        #n_correct += (pred.argmax(dim=1) == b_y).sum().item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    #train_accuracy = n_correct/len(train_dataset)
    train_avg_loss = sum(losses)/len(losses)    
    
    # Compute predictions in the validation set (with adagrad deactivated)
    # losses = []
    # n_correct = 0
    # with torch.no_grad():
    #     iter = 0
    #     for b_x, b_y in val_dataloader:
    #         pred = model(b_x)
    #         loss = loss_fn(pred, b_y)
    #         losses.append(loss.item())
            
    #         iter += 1
    #         if iter%10 == 0: 
    #             print(f'{iter} batches validated')
            
    #         if iter == 50: 
    #             break
            
    #         #hard_preds = pred.argmax(dim=1)
    #         #n_correct += (pred.argmax(dim=1) == b_y).sum().item()
    #     #val_accuracy = n_correct/len(test_dataset)
    #     val_avg_loss = sum(losses)/len(losses)      
        
        
    display_str = 'Epoch {}'
    display_str += '\tLoss: {:.3f} '
    # display_str += '\tLoss (val): {:.3f}'
    print(display_str.format(epoch, train_avg_loss)) 

In [ ]:
l, ab = example_dataset[some_random_index]
l_batch = torch.unsqueeze(l, 0)
ab_res = model.forward(l_batch).detach()
ab_res = torch.squeeze(ab_res, dim = 0)
plot_comb(l, ab_res)